<a href="https://colab.research.google.com/github/biryuchka/ml/blob/main/BLIP_chartQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=6b43f04c7419f79d629ddcfda069d68382a8e762387d87c215bd2132d40adb5b
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil
Gen RAM Free: 9.9 GB  |     Proc size: 2.1 GB
GPU RAM Free: 11MB | Used: 15091MB | Util  98% | Total     15360MB


In [1]:
!pip install datasets==3.0.1
!pip install tdqm

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1321 sha256=8639fe9910ea856b1f722b4018aa4113502f57dd54347f7d08f9234d01a12046
  Stored in directory: /root/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm


In [2]:
!pip install transformers

In [3]:
import os

import requests
from transformers import BlipProcessor, BlipForQuestionAnswering
import torch
from PIL import Image
from torch.utils.data import DataLoader
from tqdm import tqdm


In [4]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [5]:
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [6]:
from datasets import load_dataset

train_dataset = load_dataset("lmms-lab/ChartQA", split='test[:70%]')
test_dataset = load_dataset("lmms-lab/ChartQA", split='test[-30%:-10%]')
valid_dataset = load_dataset("lmms-lab/ChartQA", split='test[-10%:]')

README.md:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/72.6M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [7]:
train_dataset

Dataset({
    features: ['type', 'question', 'answer', 'image'],
    num_rows: 1750
})

In [8]:
test_dataset

Dataset({
    features: ['type', 'question', 'answer', 'image'],
    num_rows: 500
})

In [9]:
valid_dataset

Dataset({
    features: ['type', 'question', 'answer', 'image'],
    num_rows: 250
})

In [10]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]['image']
        question = self.dataset[idx]['question']
        answer = self.dataset[idx]['answer']
        encoded = processor(image, question, padding="max_length", truncation=True, return_tensors="pt")
        labels = processor.tokenizer.encode(answer, max_length= 8, pad_to_max_length=True, return_tensors='pt')
        encoded['labels'] = labels
        for k,v in encoded.items():  encoded[k] = v.squeeze()
        return encoded

In [11]:
dataset_train = MyDataset(train_dataset)
dataset_valid = MyDataset(valid_dataset)
dataset_test = MyDataset(test_dataset)

In [42]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset_train, batch_size=8, shuffle=False, pin_memory=True)
valid_loader = DataLoader(dataset_valid, batch_size=8, shuffle=False, pin_memory=True)

In [43]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import ExponentialLR

In [44]:
optimizer = torch.optim.AdamW(model.parameters(), lr=4e-5)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9, last_epoch=-1, verbose=False)
scaler = torch.cuda.amp.GradScaler()

<ipython-input-44-1a5abf6e3139>:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [45]:
patience = 10
min_eval_loss = float("inf")
early_stopping_hook = 0

In [46]:
from tqdm import tqdm


def train(model_, loader) -> float:
    model_.train()

    train_loss = 0

    for batch in tqdm(loader, desc='Train'):
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            outputs = model_(**batch)
            loss = outputs.loss

        train_loss += loss.item()
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    return train_loss / len(loader)

In [47]:
@torch.inference_mode()
def evaluate(model_, loader) -> float:
    model_.eval()

    total_loss = 0

    for batch in tqdm(loader, desc='Valid'):
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            outputs = model_(**batch)
            loss = outputs.loss

        total_loss += loss.item()
        del loss
        optimizer.zero_grad()

    total_loss /= len(loader)

    return total_loss

In [48]:
import matplotlib.pyplot as plt
from IPython.display import clear_output

def plot_stats(
    train_loss: list[float],
    valid_loss: list[float],
    title: str
):
    plt.figure(figsize=(16, 8))

    plt.title(title + ' loss')

    plt.plot(train_loss, label='Train loss')
    plt.plot(valid_loss, label='Valid loss')
    plt.legend()
    plt.grid()

    plt.show()

In [49]:
import gc
def whole_train_valid_cycle(model_, num_epochs, title):
    train_loss_history, valid_loss_history = [], []

    for epoch in range(num_epochs):
        train_loss = train(model_, train_loader)
        valid_loss = evaluate(model_, valid_loader)

        train_loss_history.append(train_loss)
        valid_loss_history.append(valid_loss)

        clear_output()

        plot_stats(
            train_loss_history, valid_loss_history,
            title
        )
        gc.collect()
        torch.cuda.empty_cache()
        scheduler.step()
        if valid_loss < min_eval_loss:
            model.save_pretrained("Model/blip-saved-model", from_pt=True)
            print("Saved model to Model/blip-saved-model")
            min_eval_loss = valid_loss
            early_stopping_hook = 0
        else:
            early_stopping_hook += 1
            if early_stopping_hook > patience:
                break


In [50]:
torch.cuda.empty_cache()
# import gc
# del variables
# gc.collect()


In [52]:
whole_train_valid_cycle(model, 100, "BLIP model on small chartQA dataset")

Train:   0%|          | 0/219 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 11.06 MiB is free. Process 12193 has 14.73 GiB memory in use. Of the allocated memory 14.44 GiB is allocated by PyTorch, and 175.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)